In [2]:
from bgen.reader import BgenFile, BgenReader
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
antonio_rsids = pd.read_table("/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/Antonino/TGHDLadjBMI/data/IR_var_TGHDL_MAGIC_nomissing.txt",header=None)

In [4]:
antonio_rsids

,0,1,2
0,1,219628973,rs6674544
1,2,630902,rs5017305
2,2,27177428,rs61007968
3,2,27730940,rs1260326
4,2,65296280,rs2252867
...,...,...,...
230,1,72835410,rs2613505
231,14,52480621,rs6572807
232,11,26281664,rs1461396
233,4,89753225,rs4425336


In [5]:
with BgenReader("/mnt/speliotes-lab/Projects/UK_ATLAS/GENODATA/BGEN_New/ukb_imp_chr1_v3.bgen", delay_parsing=True) as bfile:
    var = bfile.with_rsid("rs6674544")

In [6]:
var.alt_dosage

array([1., 2., 1., ..., 0., 2., 2.], dtype=float32)

In [7]:
chr_rsids = {}
for c,r in zip(antonio_rsids[0],antonio_rsids[2]):
    if c not in chr_rsids:
        chr_rsids[c] = set()
    chr_rsids[c].add(r)
print(len(chr_rsids))

22


In [34]:
variants = []
for chrom in range(1,23):
    print("starting chr {}".format(chrom))
    with BgenReader("/mnt/speliotes-lab/Projects/UK_ATLAS/GENODATA/BGEN_New/ukb_imp_chr{}_v3.bgen".format(chrom), delay_parsing=True) as bfile:
        rsids = bfile.rsids()
        for i,rsid in enumerate(rsids):
            if rsid in chr_rsids[chrom]:
                #print(rsid)
                var = bfile[i]
                variants.append(var)
    print("done with chr {}, len(variants) = {}".format(chrom,len(variants)))
print(len(variants))

starting chr 1
done with chr 1, len(variants) = 13
starting chr 2
done with chr 2, len(variants) = 44
starting chr 3
done with chr 3, len(variants) = 61
starting chr 4
done with chr 4, len(variants) = 78
starting chr 5
done with chr 5, len(variants) = 90
starting chr 6
done with chr 6, len(variants) = 113
starting chr 7
done with chr 7, len(variants) = 126
starting chr 8
done with chr 8, len(variants) = 140
starting chr 9
done with chr 9, len(variants) = 147
starting chr 10
done with chr 10, len(variants) = 157
starting chr 11
done with chr 11, len(variants) = 166
starting chr 12
done with chr 12, len(variants) = 188
starting chr 13
done with chr 13, len(variants) = 191
starting chr 14
done with chr 14, len(variants) = 192
starting chr 15
done with chr 15, len(variants) = 194
starting chr 16
done with chr 16, len(variants) = 200
starting chr 17
done with chr 17, len(variants) = 206
starting chr 18
done with chr 18, len(variants) = 209
starting chr 19
done with chr 19, len(variants) = 2

In [42]:
# import samples file
samples = pd.read_table("/mnt/speliotes-lab/Projects/UK_ATLAS/GENODATA/BGEN_New/ukb18120_imp_chr22_v2_s487395.sample",names = ["ID_1","ID_2","missing","sex"],skiprows=2,sep=" ")
samples

,ID_1,ID_2,missing,sex
0,2563202,2563202,0,1
1,5046258,5046258,0,2
2,1711817,1711817,0,2
3,4464917,4464917,0,2
4,5399174,5399174,0,2
...,...,...,...,...
487404,4348388,4348388,0,2
487405,5253315,5253315,0,2
487406,5539040,5539040,0,1
487407,5821524,5821524,0,2


In [47]:
# verify with a couple variants, but I believe this file is REF_FIRST
df_dict = {"eids":list(samples.ID_1)}
for i,var in enumerate(variants):
    df_dict["{}({})".format(var.rsid,"-".join([s.strip() for s in var.alleles]))] = var.alt_dosage
df = pd.DataFrame(df_dict)
#df

In [46]:
# save results to csv file
df.to_csv("Antonio_example_dosages.csv",index=False)